In [ ]:
from collections import Counter

import pymongo

import nltk
from nltk.tokenize import TweetTokenizer

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# MongoDB connection data
HOST = 'localhost'
PORT = '27017'
DB_NAME = 'trending'

### Connecting to database

In [ ]:
db = None
try:
    db_client = pymongo.MongoClient(HOST + ':' + PORT)
    db = db_client[DB_NAME]
    print('Conecction established successfully!')
except pymongo.errors.ConnectionFailure:
    print('Connection cannot be established')

In [ ]:
HISTORY_COL = db['history']

### Analizying trends (hashtags)

In [ ]:
counter = Counter()

In [ ]:
tweets = HISTORY_COL.find()

In [ ]:
tweets.count()

In [ ]:
tokenizer = TweetTokenizer(preserve_case = False, reduce_len = True)

In [ ]:
for tweet in tweets:
    try:
        terms_hash = [t for t in tokenizer.tokenize(tweet['full_text']) if (term.startswith('#') and len(term) > 1)]
        counter.update(terms_hash)
    except KeyError:
        pass

In [ ]:
trends_df = pd.DataFrame.from_dict(counter, orient = 'index').reset_index()
trends_df.columns = ['Hashtag', 'Frecuencia']
trends_df = trends_df.sort_values(by = ['Frecuencia'], ascending = False)
trends_df = trends_df.head()

In [ ]:
trends_df

In [ ]:
ax = trends_df.plot(kind = 'barh', figsize = (20, 8))
ax.set_yticklabels(trends_df['Hashtag'])

### Creating and using text index

In [ ]:
HISTORY_COL.create_index([('full_text', pymongo.TEXT)], default_language = 'spanish')

In [ ]:
tweets = HISTORY_COL.find({'$text': {'$search': 'fraude'}})

In [ ]:
tweets.count()

In [ ]:
for tweet in tweets:
    print(tweet['full_text'].replace('\n', ''))

### Analyzing time

In [ ]:
tweets = HISTORY_COL.find({}, {'created_at': True})

In [ ]:
creation_dates = []
for tweet in tweets:
    try:
        creation_dates.append(tweet['created_at'])
    except:
        pass

In [ ]:
plt.figure(figsize = (20, 8))
plt.hist(creation_dates)